# wiki3 Knowledge Graph Extraction Pipeline

Extract structured knowledge from Wikipedia articles using LLMs, generate embeddings, and prepare for browser-based querying with DuckDB-Wasm.

## Setup

In [ ]:
import { ChatOpenAI } from "npm:@langchain/openai";
import { WikipediaQueryRun } from "npm:@langchain/community/tools/wikipedia_query_run";
import { RecursiveCharacterTextSplitter } from "npm:@langchain/textsplitters";
import { ChatPromptTemplate } from "npm:@langchain/core/prompts";
import { z } from "npm:zod";

console.log("Dependencies loaded");

TypeError: [ERR_PACKAGE_PATH_NOT_EXPORTED] Package subpath './output_parsers/structured' is not defined by "exports" in '/home/jovyan/.cache/deno/npm/registry.npmjs.org/@langchain/core/1.1.6/package.json' imported from 'file:///home/jovyan/work/$deno$repl.mts'

## Wikipedia Loader & Chunking

In [ ]:
const wikipediaFetcher = new WikipediaQueryRun();
const articleTitle = "Albert Einstein";
const rawContent = await wikipediaFetcher.call(articleTitle);

console.log(`Fetched: ${articleTitle}`);
console.log(`Content length: ${rawContent.length} characters`);
console.log(`Preview: ${rawContent.substring(0, 300)}...`);

## Configure LLM for Knowledge Graph Extraction

In [ ]:
const llm = new ChatOpenAI({
  model: "gpt-4o-mini",
  temperature: 0,
  apiKey: Deno.env.get("OPENAI_API_KEY"),
});

// Define the extraction schema using Zod
const extractionSchema = z.object({
  entities: z.array(
    z.object({
      id: z.string().describe("Unique entity identifier (e.g., person_1, org_2)"),
      label: z.string().describe("Entity name or label"),
      type: z.enum(["Person", "Organization", "Place", "Concept", "Event", "Work"]),
      description: z.string().describe("Brief entity description from text"),
    })
  ).describe("List of entities extracted from the text"),
  relations: z.array(
    z.object({
      source_id: z.string().describe("Source entity ID"),
      target_id: z.string().describe("Target entity ID"),
      relation_type: z.string().describe("Type of relationship (e.g., BORN_IN, WORKED_AT, DISCOVERED)"),
      description: z.string().describe("Description of the relationship"),
    })
  ).describe("List of relationships between entities"),
});

// Use withStructuredOutput for reliable JSON extraction
const structuredLlm = llm.withStructuredOutput(extractionSchema);

console.log("LLM and schema configured with withStructuredOutput");

## Extract Knowledge Graph from Text

In [ ]:
const splitter = new RecursiveCharacterTextSplitter({
  chunkSize: 1024,
  chunkOverlap: 128,
});

const chunks = await splitter.splitText(rawContent);
console.log(`Split into ${chunks.length} chunks`);
console.log(`Chunk 1 preview: ${chunks[0].substring(0, 150)}...`);

In [ ]:
const extractionPrompt = ChatPromptTemplate.fromMessages([
  ["system", `You are an expert at extracting knowledge graphs from text.
Extract entities and relationships from the provided text.
Assign unique IDs to each entity using the format: type_number (e.g., "person_1", "org_2", "place_3").
Identify relationships between entities that appear in the same context.
Focus on factual relationships like: BORN_IN, WORKED_AT, DISCOVERED, FOUNDED, RECEIVED, MARRIED_TO, DEVELOPED, etc.`],
  ["human", "Extract entities and relationships from this text:\n\n{text}"]
]);

const extractionChain = extractionPrompt.pipe(structuredLlm);
console.log("Extraction chain created");

In [ ]:
const allEntities = new Map();
const allRelations = [];
const processChunks = 2;

for (let i = 0; i < Math.min(processChunks, chunks.length); i++) {
  console.log(`Processing chunk ${i + 1}/${Math.min(processChunks, chunks.length)}...`);
  try {
    const result = await extractionChain.invoke({
      text: chunks[i],
    });

    // Deduplicate entities by label
    for (const entity of result.entities) {
      const key = entity.label.toLowerCase();
      if (!allEntities.has(key)) {
        allEntities.set(key, {
          id: `${entity.type.toLowerCase()}_${allEntities.size + 1}`,
          label: entity.label,
          type: entity.type,
          description: entity.description,
        });
      }
    }

    // Map relations to deduplicated entity IDs
    for (const relation of result.relations) {
      const sourceEntity = result.entities.find(e => e.id === relation.source_id);
      const targetEntity = result.entities.find(e => e.id === relation.target_id);
      
      if (sourceEntity && targetEntity) {
        const sourceKey = sourceEntity.label.toLowerCase();
        const targetKey = targetEntity.label.toLowerCase();
        
        allRelations.push({
          source_id: allEntities.get(sourceKey)?.id || relation.source_id,
          target_id: allEntities.get(targetKey)?.id || relation.target_id,
          relation_type: relation.relation_type,
          description: relation.description,
        });
      }
    }

    console.log(`  Extracted ${result.entities.length} entities, ${result.relations.length} relations`);
  } catch (error) {
    console.error(`  Error processing chunk ${i + 1}:`, error.message);
  }
}

console.log(`\nTotal unique entities: ${allEntities.size}`);
console.log(`Total relations: ${allRelations.length}`);

## Inspect Extracted Knowledge Graph

In [ ]:
console.log("\n=== ENTITIES ===");
for (const [, entity] of allEntities) {
  console.log(`[${entity.id}] ${entity.label} (${entity.type})`);
  console.log(`  → ${entity.description}`);
}

In [ ]:
console.log("\n=== RELATIONS ===");
const entitiesArr = Array.from(allEntities.values());
for (const relation of allRelations) {
  const source = entitiesArr.find(e => e.id === relation.source_id);
  const target = entitiesArr.find(e => e.id === relation.target_id);
  console.log(`${source?.label} --[${relation.relation_type}]--> ${target?.label}`);
}

## Generate Embeddings

In [ ]:
import { Sha256 } from "https://deno.land/std@0.208.0/crypto/sha256.ts";

function getMockEmbedding(text) {
  const hash = new Sha256().update(text).digest();
  const embedding = [];
  for (let i = 0; i < 384; i++) {
    embedding.push((hash[i % hash.length] - 128) / 128);
  }
  return embedding;
}

const entityEmbeddings = new Map();
for (const [, entity] of allEntities) {
  entityEmbeddings.set(entity.id, {
    ...entity,
    embedding: getMockEmbedding(entity.label + entity.description),
  });
}

console.log(`Generated embeddings for ${entityEmbeddings.size} entities`);

## Export to DuckDB-Wasm Format

In [ ]:
const duckdbFormat = {
  metadata: {
    source: articleTitle,
    extracted_at: new Date().toISOString(),
    entity_count: allEntities.size,
    relation_count: allRelations.length,
  },
  entities: Array.from(entityEmbeddings.values()),
  relations: allRelations,
};

console.log("\n=== EXPORT ===");
console.log(JSON.stringify(duckdbFormat, null, 2).substring(0, 500) + "...");